In [1]:
import pandas as pd
import os
import cv2
import random

# Set up train & validation image folder for annotation format conversion

In [2]:
#import csv with bounding box labels
df = pd.read_csv('train_solution_bounding_boxes (1).csv')
df.head()

,image,xmin,ymin,xmax,ymax
0,vid_4_1000.jpg,281.259045,187.035071,327.727931,223.225547
1,vid_4_10000.jpg,15.163531,187.035071,120.329957,236.430180
2,vid_4_10040.jpg,239.192475,176.764800,361.968162,236.430180
3,vid_4_10020.jpg,496.483358,172.363256,630.020261,231.539575
4,vid_4_10060.jpg,16.630970,186.546010,132.558611,238.386422


In [3]:
df.shape

(3662, 5)

In [4]:
images = os.listdir('training_images/')

In [5]:
#create validation set of images
random.seed(4)
n = round(0.2 * len(images))
val = random.sample(images, n)
print('Number of validation images:', n)

Number of validation images: 708


In [6]:
#convert bounding box to yolo format (x_center, y_center, width, height and normalize 0 - 1)
frame = cv2.imread('training_images/' + images[0])
h, w, colour = frame.shape

df['class'] = 1 # NOTE that in our Yolo format car class id is 0, but CoCo class id should begin from 1 so we set it 1 instead


In [7]:
#Save images and labels - one text file per image
for im in images:
  frame = cv2.imread('training_images/{}'.format(im))
  df_image = df[df['image'] == im]
  df_image = df_image.drop(['image'], axis = 1)
  df_image.head()
  lbl = im[0:-3] + 'txt'
  #print(lbl)
  
  if im in val:    
    cv2.imwrite('data/valid/images/{}'.format(im), frame)
    df_image.to_csv('data/valid/labels/{}'.format(lbl), 
                    header=None, index=None, sep=' ', mode='a')
  else:
    cv2.imwrite('data/train/images/{}'.format(im), frame)
    df_image.to_csv('data/train/labels/{}'.format(lbl), 
                    header=None, index=None, sep=' ', mode='a')

# Convert Yolo to CoCo format

https://www.immersivelimit.com/tutorials/create-coco-annotations-from-scratch/#coco-dataset-format
https://www.programmersought.com/article/43131872583/

In [8]:
import json
import os
import cv2

In [9]:
train_imgs_path = 'data/train/images/'
train_labels_path = 'data/train/labels/'
test_imgs_path = 'data/valid/images/'
test_labels_path = 'data/valid/labels/'

In [10]:
def setup_annotation(imgs_path, labels_path, Phase):
    # dataset is used to save image information and annotation information of all data
    dataset = {"categories": [], "annotations": [], "images": []}

    # Add category for car for our case
    dataset["categories"].append({"id": 1, "name": "car", "supercategory": "vehicle"})

    # Read the image name of the images folder
    imgs_list = os.listdir(imgs_path)

    global anno_index
    anno_index = 0

    for i, img_name in enumerate(imgs_list):
        img = cv2.imread(imgs_path + '/{}'.format(img_name))
        h, w, _ = img.shape

        # Add image information to the dataset
        dataset['images'].append({'file_name': img_name, 'id': i, 'width': w, 'height': h})

        with open(labels_path + '/{}.txt'.format(img_name[0:-4])) as tr:
            annos = tr.readlines()

            for ii, anno in enumerate(annos):
                parts = anno.strip().split()
                # x_min
                x1 = float(parts[0])
                # y_min
                y1 = float(parts[1])
                # x_max
                x2 = float(parts[2])
                # y_max
                y2 = float(parts[3])
                class_id = parts[4]

                width = max(0, x2 - x1)
                height = max(0, y2 - y1)

                dataset['annotations'].append({
                    'area': width * height,
                    'bbox': [x1, y1, width, height],
                    'category_id': int(class_id),
                    'id': anno_index,
                    'image_id': i,
                    'iscrowd': 0,
                     # mask, the rectangle is the four vertices clockwise from the top left corner
                    'segmentation': [[x1, y1, x2, y1, x2, y2, x1, y2]]
                })

                anno_index = anno_index + 1

    #Save the resulting folder
    with open('data/annotations/{}.json'.format(Phase), 'w') as f:
      json.dump(dataset, f)

In [11]:
setup_annotation(train_imgs_path, train_labels_path, 'train')

In [12]:
setup_annotation(test_imgs_path, test_labels_path, 'test')